<a href="https://colab.research.google.com/github/davidlealo/sic_ai_2024/blob/main/007_proyecto/peliculas/%5B2%5Dmovier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hola profe no si es esto a lo que se refería pero aquí tengo algunos csv y los uní JSJSJ

In [1]:
import numpy as np
import pandas as pd

In [2]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 9.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357280 sha256=d4272c0b5e3f88d7063ff761c029c190b9462e29efe853ac6aedbde7e8d6993e
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [3]:
import os
import urllib.request
import zipfile
from surprise import KNNBasic, Dataset, Reader
from surprise import accuracy
from surprise.model_selection import train_test_split
from collections import defaultdict
import pandas as pd

In [4]:
df_muestra = pd.read_csv("df_muestra.csv")
df_muestra

,title,year_x,rotten tomatoes,netflix,hulu,prime video,disney+,movieId,genres,userId,rating
0,Ghost,2020,45/100,0,0,1,0,587.0,Comedy|Drama|Fantasy|Romance|Thriller,39481.0,4.0
1,Willow,1988,75/100,0,0,0,1,2193.0,Action|Adventure|Fantasy,110239.0,3.0
2,Ghost,2020,45/100,0,0,1,0,587.0,Comedy|Drama|Fantasy|Romance|Thriller,86400.0,4.5
3,Finding Nemo,2003,87/100,0,0,0,1,6377.0,Adventure|Animation|Children|Comedy,25669.0,4.5
4,Philadelphia,1993,79/100,0,0,1,0,508.0,Drama,55955.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...
9995,Arachnophobia,1990,70/100,0,1,0,0,2699.0,Comedy|Horror,21552.0,3.0
9996,Catch Me If You Can,2002,87/100,0,1,0,0,5989.0,Crime|Drama,88015.0,4.0
9997,My Best Friend's Wedding,1997,71/100,1,0,0,0,1569.0,Comedy|Romance,44988.0,3.0
9998,Fight Club,1999,93/100,0,0,1,0,2959.0,Action|Crime|Drama|Thriller,81510.0,4.0


In [5]:
# Crear un diccionario para mapear id de película a título
movie_id_to_title = dict(zip(df_muestra['movieId'], df_muestra['title']))

In [8]:
reader = Reader(rating_scale=(0.5, 5))  # Ajusta el rango de calificación si es diferente
data = Dataset.load_from_df(df_muestra[['userId', 'title', 'rating']], reader)

# Dividir el dataset en conjunto de entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=.25)

# Crear el algoritmo KNNBasic
modelo = KNNBasic(k=40, sim_options={'name': 'cosine', 'user_based': True})

# Entrenar el algoritmo con el conjunto de entrenamiento
modelo.fit(trainset)

# Hacer predicciones en el conjunto de prueba
predictions = modelo.test(testset)

# Calcular el RMSE (Root Mean Squared Error)
rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse}")

# Función para obtener las top N recomendaciones
def get_top_n(predictions, n=5):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Ordenar las predicciones para cada usuario y obtener las top n
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return dict(top_n)

# Obtener las top 5 recomendaciones para todos los usuarios
top_n = get_top_n(predictions, n=5)

# Mostrar las top 5 recomendaciones para el primer usuario disponible
first_user_id = list(top_n.keys())[0]  # Obtener el primer usuario disponible
print(f"\nTop 5 recomendaciones para el usuario {first_user_id}:")
for iid, rating in top_n[first_user_id]:
    # Si iid ya es un título, no hace falta convertirlo a entero
    pelicula = movie_id_to_title.get(int(iid), iid) if iid.isdigit() else iid
    print(f"Película: {pelicula}, Calificación estimada: {rating:.2f}")

# Mostrar todas las predicciones para el primer usuario disponible
print(f"\nTodas las predicciones para el usuario {first_user_id}:")
user_predictions = [pred for pred in predictions if pred.uid == first_user_id]
for prediction in sorted(user_predictions, key=lambda x: x.est, reverse=True)[:10]:
    pelicula = movie_id_to_title.get(int(prediction.iid), prediction.iid) if prediction.iid.isdigit() else prediction.iid
    print(f"Película: {pelicula}, Calificación estimada: {prediction.est:.2f}")


Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0229
RMSE: 1.0229059869754515

Top 5 recomendaciones para el usuario 94976.0:
Película: Spanglish, Calificación estimada: 3.51

Todas las predicciones para el usuario 94976.0:
Película: Spanglish, Calificación estimada: 3.51


In [9]:
# Mostrar las top 5 recomendaciones para el primer usuario disponible
first_user_id = list(top_n.keys())[0]  # Obtener el primer usuario disponible
print(f"\nTop 5 recomendaciones para el usuario {first_user_id}:")
for iid, rating in top_n[first_user_id]:
    # Buscar el título de la película usando el movieId en lugar de convertir a int
    pelicula = df_muestra[df_muestra['movieId'] == float(iid)]['title'].values[0] if iid.isdigit() else iid
    print(f"Película: {pelicula}, Calificación estimada: {rating:.2f}")

# Mostrar todas las predicciones para el primer usuario disponible
print(f"\nTodas las predicciones para el usuario {first_user_id}:")
user_predictions = [pred for pred in predictions if pred.uid == first_user_id]
for prediction in sorted(user_predictions, key=lambda x: x.est, reverse=True)[:10]:
    pelicula = df_muestra[df_muestra['movieId'] == float(prediction.iid)]['title'].values[0] if prediction.iid.isdigit() else prediction.iid
    print(f"Película: {pelicula}, Calificación estimada: {prediction.est:.2f}")



Top 5 recomendaciones para el usuario 94976.0:
Película: Spanglish, Calificación estimada: 3.51

Todas las predicciones para el usuario 94976.0:
Película: Spanglish, Calificación estimada: 3.51


In [10]:
import random
from IPython.display import Javascript, display, HTML
from google.colab.output import eval_js
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD

# Asumimos que df_muestra ya está cargado con las películas y ratings

# Convertir el DataFrame en el formato esperado por Surprise
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(df_muestra[['userId', 'movieId', 'rating']], reader)

# Entrenar el modelo con Surprise
trainset = data.build_full_trainset()
modelo_svd = SVD()
modelo_svd.fit(trainset)

# Función para obtener recomendaciones de películas basadas en predicciones
def get_top_movies_for_user(user_id, last_liked_movie_id=None, n=5):
    # Obtener todas las películas que el usuario no ha visto
    unseen_movies = set(df_muestra['movieId']) - set(df_muestra[df_muestra['userId'] == user_id]['movieId'])

    # Predecir ratings para estas películas usando el modelo de Surprise
    predictions = [(movie_id, modelo_svd.predict(user_id, movie_id).est) for movie_id in unseen_movies]

    # Si hay una película que le gustó, filtrar por similitud
    if last_liked_movie_id is not None:
        # Filtrar las películas que tengan un score más cercano a la última película que le gustó
        similar_movies = [(movie_id, score) for movie_id, score in predictions if abs(score - modelo_svd.predict(user_id, last_liked_movie_id).est) < 0.1]
        predictions = similar_movies if similar_movies else predictions  # Si no hay similares, usar todas

    # Ordenar por rating predicho y devolver las top n
    return sorted(predictions, key=lambda x: x[1], reverse=True)[:n]

# Inicializar lista de películas recomendadas para un usuario aleatorio
def inicializar_recomendaciones():
    global user_id, recommended_movies, current_index, last_liked_movie_id
    user_id = random.choice(df_muestra['userId'].unique())  # Seleccionar un usuario al azar
    recommended_movies = get_top_movies_for_user(user_id)
    current_index = 0
    last_liked_movie_id = None  # Reiniciar la última película que le gustó

inicializar_recomendaciones()  # Generar la primera recomendación

liked_movies = []  # Lista para almacenar películas que le gustan
like_count = 0  # Contador de "me gusta"

def create_buttons(movie_title):
    movie_title_escaped = movie_title.replace("'", "\'").replace('"', '\"')  # Escapar caracteres
    display(Javascript(f'''
        var div = document.createElement('div');
        var btn1 = document.createElement('button');
        btn1.innerHTML = 'Me gusta: {movie_title_escaped}';
        btn1.onclick = function() {{
          google.colab.kernel.invokeFunction('notebook.callback', ['Me gusta', '{movie_title_escaped}'], {{}});
        }};

        var btn2 = document.createElement('button');
        btn2.innerHTML = 'No me gusta: {movie_title_escaped}';
        btn2.onclick = function() {{
          google.colab.kernel.invokeFunction('notebook.callback', ['No me gusta', '{movie_title_escaped}'], {{}});
        }};

        div.appendChild(btn1);
        div.appendChild(btn2);
        document.body.appendChild(div);
    '''))

def handle_choice(choice, movie_title):
    global user_id, recommended_movies, current_index, last_liked_movie_id, liked_movies, like_count

    print(f'Has elegido: {choice} para {movie_title}')

    if choice == 'Me gusta':
        # Obtener el ID de la película
        movie_id = df_muestra[df_muestra['title'] == movie_title]['movieId'].values[0]

        # Guardar la última película que le gustó
        last_liked_movie_id = movie_id
        liked_movies.append(movie_title)  # Guardar título de la película
        like_count += 1  # Incrementar el contador de "me gusta"

        # Verificar si ha alcanzado 6 "me gusta"
        if like_count >= 6:
            # Crear el mensaje final con las películas que le gustaron
            final_message = "Te gustan las siguientes películas:\n"
            final_message += ', '.join(liked_movies)

            display(HTML(final_message))
            return  # Termina el programa

        current_index += 1
        if current_index < len(recommended_movies):
            next_movie_id, next_score = recommended_movies[current_index]
            next_movie_title = df_muestra[df_muestra['movieId'] == next_movie_id]['title'].values[0]
            print(f"Película recomendada: {next_movie_title}, score: {next_score}")
            create_buttons(next_movie_title)
        else:
            print("No hay más películas en la lista de recomendaciones. Seleccionando un nuevo usuario.")
            inicializar_recomendaciones()
            next_movie_id, next_score = recommended_movies[current_index]
            next_movie_title = df_muestra[df_muestra['movieId'] == next_movie_id]['title'].values[0]
            print(f"Nuevo usuario seleccionado. Película recomendada: {next_movie_title}, score: {next_score}")
            create_buttons(next_movie_title)
    else:
        # Seleccionar una nueva película si elige 'No me gusta'
        unseen_movies = set(df_muestra['movieId']) - set(df_muestra[df_muestra['userId'] == user_id]['movieId'])
        next_movie_id = random.choice(list(unseen_movies))
        next_movie_title = df_muestra[df_muestra['movieId'] == next_movie_id]['title'].values[0]
        next_score = modelo_svd.predict(user_id, next_movie_id).est
        print(f"Nueva película recomendada: {next_movie_title}, score: {next_score}")
        create_buttons(next_movie_title)

# Registrar la función en Google Colab
from google.colab import output
output.register_callback('notebook.callback', handle_choice)

# Mostrar la primera película recomendada
first_movie_id, first_score = recommended_movies[current_index]
first_movie_title = df_muestra[df_muestra['movieId'] == first_movie_id]['title'].values[0]
print(f"Película recomendada: {first_movie_title}, score: {first_score}")
create_buttons(first_movie_title)


Película recomendada: Rebecca, score: 4.207787318423066


<IPython.core.display.Javascript object>

Has elegido: No me gusta para Rebecca
Nueva película recomendada: Big Fish, score: 3.8186110538952


<IPython.core.display.Javascript object>

Has elegido: Me gusta para Big Fish
Película recomendada: There Will Be Blood, score: 4.164938543488736


...

<IPython.core.display.Javascript object>

Has elegido: Me gusta para Fight Club
No hay más películas en la lista de recomendaciones. Seleccionando un nuevo usuario.
Nuevo usuario seleccionado. Película recomendada: Taxi Driver, score: 4.201978531742617


<IPython.core.display.Javascript object>

Has elegido: Me gusta para Taxi Driver


Has elegido: Me gusta para Taxi Driver


In [11]:
import random
from IPython.display import Javascript, display, HTML
from google.colab.output import eval_js
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD

# Asumimos que df_muestra ya está cargado con las películas y ratings

# Convertir el DataFrame en el formato esperado por Surprise
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(df_muestra[['userId', 'movieId', 'rating']], reader)

# Entrenar el modelo con Surprise
trainset = data.build_full_trainset()
modelo_svd = SVD()
modelo_svd.fit(trainset)

# Función para obtener recomendaciones de películas basadas en predicciones
def get_top_movies_for_user(user_id, last_liked_movie_id=None, n=5):
    # Obtener todas las películas que el usuario no ha visto
    unseen_movies = set(df_muestra['movieId']) - set(df_muestra[df_muestra['userId'] == user_id]['movieId'])

    # Predecir ratings para estas películas usando el modelo de Surprise
    predictions = [(movie_id, modelo_svd.predict(user_id, movie_id).est) for movie_id in unseen_movies]

    # Si hay una película que le gustó, filtrar por similitud
    if last_liked_movie_id is not None:
        # Filtrar las películas que tengan un score más cercano a la última película que le gustó
        similar_movies = [(movie_id, score) for movie_id, score in predictions if abs(score - modelo_svd.predict(user_id, last_liked_movie_id).est) < 0.1]
        predictions = similar_movies if similar_movies else predictions  # Si no hay similares, usar todas

    # Ordenar por rating predicho y devolver las top n
    return sorted(predictions, key=lambda x: x[1], reverse=True)[:n]

# Inicializar lista de películas recomendadas para un usuario aleatorio
def inicializar_recomendaciones():
    global user_id, recommended_movies, current_index, last_liked_movie_id
    user_id = random.choice(df_muestra['userId'].unique())  # Seleccionar un usuario al azar
    recommended_movies = get_top_movies_for_user(user_id)
    current_index = 0
    last_liked_movie_id = None  # Reiniciar la última película que le gustó

inicializar_recomendaciones()  # Generar la primera recomendación

liked_movies = []  # Lista para almacenar películas que le gustan
like_count = 0  # Contador de "me gusta"

def create_buttons(movie_title):
    movie_title_escaped = movie_title.replace("'", "\'").replace('"', '\"')  # Escapar caracteres
    display(Javascript(f'''
        var div = document.createElement('div');
        var btn1 = document.createElement('button');
        btn1.innerHTML = 'Me gusta: {movie_title_escaped}';
        btn1.onclick = function() {{
          google.colab.kernel.invokeFunction('notebook.callback', ['Me gusta', '{movie_title_escaped}'], {{}});
        }};

        var btn2 = document.createElement('button');
        btn2.innerHTML = 'No me gusta: {movie_title_escaped}';
        btn2.onclick = function() {{
          google.colab.kernel.invokeFunction('notebook.callback', ['No me gusta', '{movie_title_escaped}'], {{}});
        }};

        div.appendChild(btn1);
        div.appendChild(btn2);
        document.body.appendChild(div);
    '''))

def handle_choice(choice, movie_title):
    global user_id, recommended_movies, current_index, last_liked_movie_id, liked_movies, like_count

    print(f'Has elegido: {choice} para {movie_title}')

    if choice == 'Me gusta':
        # Obtener el ID de la película
        movie_id = df_muestra[df_muestra['title'] == movie_title]['movieId'].values[0]

        # Guardar la última película que le gustó
        last_liked_movie_id = movie_id
        liked_movies.append(movie_title)  # Guardar título de la película
        like_count += 1  # Incrementar el contador de "me gusta"

        # Verificar si ha alcanzado 6 "me gusta"
        if like_count >= 6:
            # Crear el mensaje final con las películas que le gustaron
            final_message = "<h3>Te gustan las siguientes películas:</h3>"
            final_message += '<ul>' + ''.join(f"<li>{movie}</li>" for movie in liked_movies) + '</ul>'

            # Contar la frecuencia de las plataformas
            platform_counts = {
                'Netflix': sum(df_muestra[df_muestra['title'] == movie]['netflix'].values[0] for movie in liked_movies),
                'Hulu': sum(df_muestra[df_muestra['title'] == movie]['hulu'].values[0] for movie in liked_movies),
                'Prime Video': sum(df_muestra[df_muestra['title'] == movie]['prime video'].values[0] for movie in liked_movies),
                'Disney+': sum(df_muestra[df_muestra['title'] == movie]['disney+'].values[0] for movie in liked_movies)
            }

            # Determinar la plataforma más frecuente
            best_platform = max(platform_counts, key=platform_counts.get)

            # Resaltar en rojo la mejor plataforma
            final_message += f"<h4 style='color: red;'>La plataforma que más se ajusta a tus gustos es: {best_platform}</h4>"

            display(HTML(final_message))
            return  # Termina el programa

        current_index += 1
        if current_index < len(recommended_movies):
            next_movie_id, next_score = recommended_movies[current_index]
            next_movie_title = df_muestra[df_muestra['movieId'] == next_movie_id]['title'].values[0]
            print(f"Película recomendada: {next_movie_title}, score: {next_score}")
            create_buttons(next_movie_title)
        else:
            print("No hay más películas en la lista de recomendaciones. Seleccionando un nuevo usuario.")
            inicializar_recomendaciones()
            next_movie_id, next_score = recommended_movies[current_index]
            next_movie_title = df_muestra[df_muestra['movieId'] == next_movie_id]['title'].values[0]
            print(f"Nuevo usuario seleccionado. Película recomendada: {next_movie_title}, score: {next_score}")
            create_buttons(next_movie_title)
    else:
        # Seleccionar una nueva película si elige 'No me gusta'
        unseen_movies = set(df_muestra['movieId']) - set(df_muestra[df_muestra['userId'] == user_id]['movieId'])
        next_movie_id = random.choice(list(unseen_movies))
        next_movie_title = df_muestra[df_muestra['movieId'] == next_movie_id]['title'].values[0]
        next_score = modelo_svd.predict(user_id, next_movie_id).est
        print(f"Nueva película recomendada: {next_movie_title}, score: {next_score}")
        create_buttons(next_movie_title)

# Registrar la función en Google Colab
from google.colab import output
output.register_callback('notebook.callback', handle_choice)

# Mostrar la primera película recomendada
first_movie_id, first_score = recommended_movies[current_index]
first_movie_title = df_muestra[df_muestra['movieId'] == first_movie_id]['title'].values[0]
print(f"Película recomendada: {first_movie_title}, score: {first_score}")
create_buttons(first_movie_title)


Película recomendada: WALL·E, score: 4.058475076645252


<IPython.core.display.Javascript object>

Has elegido: Me gusta para WALL·E
Película recomendada: Layer Cake, score: 3.9895675766269627


<IPython.core.display.Javascript object>

Has elegido: No me gusta para Layer Cake
Nueva película recomendada: Rent, score: 3.4210814929780624


...

Has elegido: No me gusta para Super Troopers
Nueva película recomendada: Hellboy, score: 3.2634273692741553


<IPython.core.display.Javascript object>

Has elegido: Me gusta para Hellboy
No hay más películas en la lista de recomendaciones. Seleccionando un nuevo usuario.
Nuevo usuario seleccionado. Película recomendada: Fight Club, score: 4.290713955088835


<IPython.core.display.Javascript object>

Has elegido: Me gusta para Fight Club
